In [1]:
#Alternating Least Square
#recommender system algorithm
#noyau plante lorsque l'algo tourne, ou ordi pleure. voir journal de bord en bas de page

In [83]:
#Création matrice modèle
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from scipy import sparse
from numpy.linalg import inv
from time import time
import matplotlib.pyplot as plt

In [84]:
#Create functions


#Fonctions d'évaluation
def get_rmse(pred, actual):
    # Ignore non zero terms.
    pred = pred[actual.nonzero()].flatten()
    actual = actual[actual.nonzero()].flatten()
    return np.sqrt(mean_squared_error(pred, actual))


#Script algo ALS avec nombre d'itérations différentes, pour définir le meilleur nombre d'itérations
#On construit la courbe des RMSE train et test en fonction du nombre d'itérations de l'algo

def calculate_learning_curve_als(matrice_modele, matrice_test, iter_array, user_reg, movie_reg, n_factors):   
    iter_array.sort()
    train_rmse =[]
    test_rmse = []
    for (i, n_iter) in enumerate(iter_array):
        predictions = MovieRecommender_als(matrice_modele, n_factors = n_factors, n_iter = n_iter,
                             user_reg = user_reg, movie_reg = movie_reg)
        train_rmse += [get_rmse(predictions, matrice_modele)]
        test_rmse += [get_rmse(predictions, matrice_test)]
    return train_rmse, test_rmse

#Fonction d'affichage
def plot_learning_curve(iter_array, model):
    plt.figure()
    plt.plot(iter_array, model[0], label='Training', linewidth=5)
    plt.plot(iter_array, model[1], label='Test', linewidth=5)
    plt.xticks(fontsize=16);
    plt.yticks(fontsize=16);
    plt.xlabel('iterations', fontsize=30);
    plt.ylabel('RMSE', fontsize=30);
    plt.legend(loc='best', fontsize=20);
    
    

#Fonction Algorithme complet
def MovieRecommender_als(matrice_modele,n_factors,n_iter,user_reg,movie_reg):
    n_users,n_movies=matrice_modele.shape
    X,Y=initialisation(n_users,n_movies,n_factors)
    for i in range(n_iter):
        X=user_step(X,Y,matrice_modele,user_reg)
        Y=movie_step(X,Y,matrice_modele,movie_reg)
    mat_pred=get_pred(X,Y)
    return mat_pred

def movie_step(X,Y,matrice_modele,movie_reg):
    identite=np.eye
    Y=matrice_modele.T.dot(X).dot(inv((X.T).dot(X)+movie_reg*identite(Y.shape[1])))
    return Y


#Etape 1 : initialisation
def initialisation(n_users, n_movies, n_factors):
    X=np.random.random((n_users,n_factors))
    Y=np.random.random((n_movies,n_factors))
    return X,Y

#Etape 2 : Calcul des vecteurs latents pendant une itération
def user_step(X,Y,matrice_modele,user_reg):
    identite=np.eye
    X=matrice_modele.dot(Y).dot(inv((Y.T).dot(Y)+user_reg*identite(X.shape[1])))
    return X  

#Etape 3 : Prédictions
def get_pred(X,Y):
    return X.dot(Y.T)






In [85]:
#Chargement des df ratings et df qui contient conversion entre movielens et imdb
ratings=pd.read_csv('../../new_dataset/action_movies.csv',index_col=0)
movie_similarity=np.loadtxt('../../../../../dataset/movie_similarity_action.csv')

ratings.rating=ratings.rating/5
ratings=ratings.sort_values(by='movie_Action')
ratings.head()
ratings.describe()

,userId,Id_Action,movieId_ref,rating,movie_Action
count,442548.000000,442548.000000,442548.000000,442548.000000,442548.000000
mean,69278.121212,560.793543,5268.435033,0.622789,449.735522
std,39265.684746,339.012901,4739.926185,0.203563,294.085097
min,116.000000,0.000000,9.000000,0.100000,0.000000
25%,35004.000000,258.000000,1607.000000,0.500000,190.000000
50%,69378.000000,554.000000,3606.000000,0.600000,411.000000
75%,102911.000000,856.000000,7566.000000,0.800000,698.000000
max,138397.000000,1167.000000,21945.000000,1.000000,1060.000000


In [86]:
#Définition du nombre de users et du nombre de films
n_users = ratings.Id_Action.unique().shape[0]
n_movies = ratings.movie_Action.unique().shape[0]
print ("Nombre d'utilisateurs : " + str(n_users) + " | Nombre de films : " + str(n_movies))

Nombre d'utilisateurs : 1168 | Nombre de films : 1061


In [87]:
#séparer jeu de test et jeu d entrainement

train_data, test_data=train_test_split(ratings, test_size=0.2)

#del ratings


offset=0
# Création de 2 matrices modèles train et test

matrice_train=np.zeros((n_users, n_movies))
for line in train_data.itertuples():
    matrice_train[line[2]-offset, line[5]-offset]=line[4]

del train_data
    
matrice_test=np.zeros((n_users, n_movies))
for line in test_data.itertuples():
    matrice_test[line[2]-offset, line[5]-offset]=line[4]
    
del test_data


#Création matrice creuse

matrice_train_creuse=sparse.csr_matrix(matrice_train)
matrice_test_creuse=sparse.csr_matrix(matrice_test)

In [89]:
# Essai pour un nombre unique d'itération

t0=time()
pred=MovieRecommender_als(matrice_modele=matrice_train_creuse,n_factors=3,n_iter=6,user_reg=0,movie_reg=0)
rmse_train=get_rmse(pred,matrice_train)
rmse_test=get_rmse(pred,matrice_test)
t1=time()
print("rmse_train",rmse_train)
print("rmse_test",rmse_test)
print("time :", t1-t0)

rmse_train 0.3730831747671846
rmse_test 0.37743352444687933
time : 0.2828669548034668
